In [20]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [21]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [22]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [23]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    441     442 

## Models

In [24]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [25]:

model_monmlp <- train(x=TrainData, y=TrainClasses, 
               method = "monmlp") 

** Ensemble 1 
0.2488699 
** 0.2488699 

** Ensemble 1 
0.01779831 
** 0.01779831 

** Ensemble 1 
0.02394965 
** 0.02394965 

** Ensemble 1 
Complex eigenvalues found for method = BFGS 
coefficients for function value 0.3101063  :
  [1]  -72.92877498 -197.78481741   24.49044385 -136.30312957 -238.24687821
  [6]  243.27053787  261.02860441  -44.39076808   44.37912225  -84.68599544
 [11]   93.88897153   27.05063200   60.65116108  -67.22584379  -34.92863657
 [16]   34.99163082  -91.65584162   40.88430293   47.44099284  -26.03401631
 [21]   12.79578133  -12.77169609   77.29019303  -49.89348496  -36.07077328
 [26]  -19.42035927  -10.55364294  -86.10086232   86.11921023    3.43866792
 [31]   -3.52942198   69.74188614  -69.68878235   71.79923574  -99.94583785
 [36]   17.79843209  105.83426374  -59.46959008  -13.02792665   13.04937562
 [41]  -80.80265628   -4.98977552  -24.97717718  124.07745025  146.93708435
 [46]  -93.64721747  102.47573317   38.13690817   47.92763769   12.79151463
 [51]  1

In [26]:

model_mlpW <- train(x=TrainData, y=TrainClasses, 
               method = "mlpWeightDecay") 

## Results

In [27]:
print(model_monmlp)
plot(model_monmlp,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Monotone Multi-Layer Perceptron Neural Network 

883 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 883, 883, 883, 883, 883, 883, ... 
Resampling results across tuning parameters:

  hidden1  Accuracy   Kappa    
  1        0.7174313  0.4342886
  3        0.7126387  0.4250883
  5        0.6904801  0.3814173

Tuning parameter 'n.ensemble' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were hidden1 = 1 and n.ensemble = 1.


In [28]:
print(model_mlpW)
plot(model_mlpW,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Multi-Layer Perceptron 

883 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 883, 883, 883, 883, 883, 883, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.7292168  0.4586502
  1     1e-04  0.7266229  0.4534865
  1     1e-01  0.4898814  0.0000000
  3     0e+00  0.7364072  0.4724219
  3     1e-04  0.7447141  0.4884522
  3     1e-01  0.4977433  0.0000000
  5     0e+00  0.7430044  0.4858416
  5     1e-04  0.7422231  0.4845444
  5     1e-01  0.4946801  0.0000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 3 and decay = 1e-04.


In [29]:
# collect resamples
results <- resamples(list(monmlp=model_monmlp,  mlpWeightDecay=model_mlpW))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: monmlp, mlpWeightDecay 
Number of resamples: 25 

Accuracy 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.6677524 0.6923077 0.7213622 0.7174313 0.7325228 0.7737003    0
mlpWeightDecay 0.7142857 0.7303030 0.7398119 0.7447141 0.7623457 0.7959866    0

Kappa 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.3372137 0.3832331 0.4409104 0.4342886 0.4651197 0.5472439    0
mlpWeightDecay 0.4194332 0.4569759 0.4748656 0.4884522 0.5241474 0.5916307    0


## Prediction

In [30]:
pred_mlpW <- predict(model_mlpW,TestData)
pred_mlpW.prob <- predict(model_mlpW,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_mlpW) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      55      72
   NotFast   52     200
                                          
               Accuracy : 0.6728          
                 95% CI : (0.6231, 0.7199)
    No Information Rate : 0.7177          
    P-Value [Acc > NIR] : 0.97584         
                                          
                  Kappa : 0.2359          
                                          
 Mcnemar's Test P-Value : 0.08796         
                                          
            Sensitivity : 0.5140          
            Specificity : 0.7353          
         Pos Pred Value : 0.4331          
         Neg Pred Value : 0.7937          
             Prevalence : 0.2823          
         Detection Rate : 0.1451          
   Detection Prevalence : 0.3351          
      Balanced Accuracy : 0.6247          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5140187            0.7352941            0.4330709 
      Neg Pred Value            Precision               Recall 
           0.7936508            0.4330709            0.5140187 
                  F1           Prevalence       Detection Rate 
           0.4700855            0.2823219            0.1451187 
Detection Prevalence    Balanced Accuracy 
           0.3350923            0.6246564

In [31]:
roc_obj <- multiclass.roc(TestClasses, as.numeric(pred_mlpW))
roc_obj


Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_mlpW))

Data: as.numeric(pred_mlpW) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6134

In [32]:
pred_mlpW.prob

,fast,NotFast
19,0.90790445,0.09209552
24,0.03585894,0.96414107
25,0.16410400,0.83589602
27,0.39299235,0.60700768
28,0.55790371,0.44209629
31,0.17745149,0.82254857
35,0.08869272,0.91130728
36,0.23289633,0.76710367
37,0.15575506,0.84424496
41,0.06674353,0.93325651
